# Architectural Patterns

This notebook contains snippets used to illustrate the various patterns

In [5]:
%pip install -qU langchain-google-vertexai

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires kubernetes<27,>=8.0.0, but you have kubernetes 30.1.0 which is incompatible.
kfp 2.5.0 requires urllib3<2.0.0, but you have urllib3 2.2.1 which is incompatible.
langchain 0.2.11 requires langchain-core<0.3.0,>=0.2.23, but you have langchain-core 0.3.6 which is incompatible.
langchain-chroma 0.1.2 requires langchain-core<0.3,>=0.1.40, but you have langchain-core 0.3.6 which is incompatible.
langchain-community 0.2.10 requires langchain-core<0.3.0,>=0.2.23, but you have langchain-core 0.3.6 which is incompatible.
langchain-google-genai 1.0.8 requires google-generativeai<0.8.0,>=0.7.0, but you have google-generativeai 0.8.1 which is incompatible.
langchain-google-genai 1.0.8 requires langchain-core<0.3,>=0.2.17, but you have langchain-core 0.3.6 which is incompa

In [1]:
from dotenv import load_dotenv

load_dotenv("../genai_agents/keys.env");

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_google_vertexai import ChatVertexAI
from langchain_core.output_parsers import StrOutputParser

## 1. Generate each time

In [12]:
model = ChatVertexAI(model="gemini-1.5-flash")
prompt_template = PromptTemplate.from_template(
    """
    You are an AI executive assistant to {sender_name} who writes letters on behalf of the executive.
    Write a 3-5 sentence thank you message to {recipient_name} for {reason_for_thanks}.
    Extract the first name from {sender_name} and sign the message with just the first name.
    """
)
parser = StrOutputParser()
chain = prompt_template | model | parser

response = chain.invoke({
    "recipient_name": "John Doe",
    "reason_for_thanks": "speaking at our Data Conference",
    "sender_name": "Jane Brown",
})

print(response)

Dear John,

On behalf of everyone at [Company Name], I wanted to extend our sincere gratitude for your engaging and insightful presentation at the Data Conference. Your expertise on [topic] was invaluable, and we greatly appreciate you sharing your knowledge with us. 

Sincerely, 

Jane 



## 2. Cache responses

In [18]:
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache

set_llm_cache(InMemoryCache())

prompt_template = PromptTemplate.from_template(
    """
    What are the steps to put a freeze on my credit card account?
    """
)
chain = prompt_template | model | parser

First time

In [19]:
%%time
print(chain.invoke({}))

I can't provide specific steps for freezing your credit card account. That's because:

* **Each credit card issuer has its own process.** There isn't a universal method for freezing accounts. You'll need to contact your specific credit card company directly.
* **"Freezing" a credit card usually means closing it.** It's very unlikely that a credit card issuer would offer a temporary freeze. If you need to stop using your card temporarily, you can simply cut it up or keep it in a safe place. 

**However, I can provide general guidance on what you should do:**

1. **Contact your credit card issuer.**  Look for a phone number or website address on your card or your statement.
2. **Explain your reason for wanting to close the account.** Be honest and clear about your situation.
3. **Follow their instructions.** They may ask for specific information or request you to submit a form.
4. **Confirm the closure.** Make sure you understand the process and receive confirmation that your account is 

Second time

In [20]:
%%time
print(chain.invoke({}))

I can't provide specific steps for freezing your credit card account. That's because:

* **Each credit card issuer has its own process.** There isn't a universal method for freezing accounts. You'll need to contact your specific credit card company directly.
* **"Freezing" a credit card usually means closing it.** It's very unlikely that a credit card issuer would offer a temporary freeze. If you need to stop using your card temporarily, you can simply cut it up or keep it in a safe place. 

**However, I can provide general guidance on what you should do:**

1. **Contact your credit card issuer.**  Look for a phone number or website address on your card or your statement.
2. **Explain your reason for wanting to close the account.** Be honest and clear about your situation.
3. **Follow their instructions.** They may ask for specific information or request you to submit a form.
4. **Confirm the closure.** Make sure you understand the process and receive confirmation that your account is 

<b> 2 seconds -> 2 milliseconds </b> because of the cache.

## 3. Pregenerated templates

In [22]:
prompt_template = PromptTemplate.from_template(
    """
    Write a letter to a customer who has purchased a tour package.
    The customer is traveling {group_type} and the tour is to {destination}.
    Sound excited to see them and explain some of the highlights of what they will see there
    and some of the things they can do while there.
    In the letter, use [CUSTOMER_NAME] to indicate the place to be replaced by their name
    and [TOUR_GUIDE] to indicate the place to be replaced by the name of the tour guide.
    """
)
chain = prompt_template | model | parser
print(chain.invoke({
    "group_type": "family",
    "destination": "Toledo, Spain",
}))

Dear [CUSTOMER_NAME],

We are thrilled to welcome you to Toledo on your upcoming tour! We can't wait to show you the beauty and history of this enchanting city.

Toledo, known as the "City of Three Cultures," boasts a fascinating blend of Christian, Muslim, and Jewish heritage.  You'll be mesmerized by the stunning architecture, from the imposing Alcázar fortress to the majestic Toledo Cathedral. 

During your tour, you'll have the opportunity to:

* **Explore the historic Jewish Quarter:** Wander through the narrow streets lined with ancient synagogues and traditional houses. 
* **Visit the Monastery of San Juan de los Reyes:** Admire the exquisite Gothic architecture and stunning cloisters.
* **Experience the panoramic views:** Take a scenic walk along the banks of the Tagus River and soak in the breathtaking views of the city.
* **Delve into the art of Toledo:** Discover the works of El Greco, the renowned painter who captured the essence of this city in his art.

Our expert tour gu